## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,83.79,89,100,24.61,moderate rain
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,-22.00,69,0,14.97,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,77.16,94,100,18.41,light rain
3,3,Develi,TR,38.3906,35.4922,24.15,85,82,2.39,broken clouds
4,4,Borborema,BR,-21.6197,-49.0736,70.32,95,86,4.18,heavy intensity rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your destination? '))
max_temp = float(input('What is the maximum temperature you would like for your destination? '))

What is the minimum temperature you would like for your destination? 75
What is the maximum temperature you would like for your destination? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
requested_temps = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                (city_data_df["Max Temp"] >= min_temp)]
requested_temps

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,83.79,89,100,24.61,moderate rain
2,2,Vaini,TO,-21.2000,-175.2000,77.16,94,100,18.41,light rain
14,14,Tual,ID,-5.6667,132.7500,83.80,77,100,16.60,overcast clouds
15,15,Avarua,CK,-21.2078,-159.7750,80.65,78,94,13.80,overcast clouds
16,16,Puerto Escondido,MX,15.8500,-97.0667,77.83,78,0,0.00,clear sky
...,...,...,...,...,...,...,...,...,...,...
641,641,Sibolga,ID,1.7427,98.7792,83.75,69,100,3.71,overcast clouds
643,643,Poindimié,NC,-20.9333,165.3333,84.69,76,96,10.76,overcast clouds
651,651,Itarema,BR,-2.9248,-39.9167,78.48,80,49,10.49,light rain
653,653,Beira,MZ,-19.8436,34.8389,82.49,78,75,10.36,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
requested_temps.count()

City_ID                147
City                   147
Country                147
Lat                    147
Lng                    147
Max Temp               147
Humidity               147
Cloudiness             147
Wind Speed             147
Weather Description    147
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#dropna() is needed when empty rows but this DF has none. 

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = requested_temps[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,83.79,moderate rain,-21.3585,55.7679,
2,Vaini,TO,77.16,light rain,-21.2000,-175.2000,
14,Tual,ID,83.80,overcast clouds,-5.6667,132.7500,
15,Avarua,CK,80.65,overcast clouds,-21.2078,-159.7750,
16,Puerto Escondido,MX,77.83,clear sky,15.8500,-97.0667,
24,The Valley,AI,75.09,overcast clouds,18.2170,-63.0578,
31,Rikitea,PF,78.98,clear sky,-23.1203,-134.9692,
32,Arraial do Cabo,BR,75.49,overcast clouds,-22.9661,-42.0278,
33,Cayenne,GF,79.09,light rain,4.9333,-52.3333,
34,Saint-Pierre,RE,83.88,overcast clouds,-21.3393,55.4781,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
#all rows have a hotel name so no rows need to be dropped.

City                   147
Country                147
Max Temp               147
Weather Description    147
Lat                    147
Lng                    147
Hotel Name             147
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'


# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Maximum Temp</dt><dd>{Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [15]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))